![](ubc_header.png)

# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part I: Data Preprocessing

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


****

## Load Data Files

### Set Data File Path

In [4]:
# Select data file path for the chosen venue and time range where the recipes data stored

# OK 2023
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "OK", "*.oc"))

# Totem 2023
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Feast 23-24 Sep-Dec*", "*.oc"))

# Gather 2023
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Gather", "*.oc"))

# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "BLUEBERRY_FESTIVAL_24"))

filepath_list

['/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/OK/OK_Menu_23_07_2024.oc']

### Import Items List

In [5]:
# Read items.xml files in the filepath_list and construct a dataframe
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []


# from the items xml file, findtext of CaseQty, CaseUOM, PakQty, PakUOM, and InventoryGroup
# then append it on the lists above

for filepath in filepath_list:
    path = filepath + '/Items.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for item in xtree.iterfind('Item'):
            ItemId.append(item.attrib['id'])
            Description.append(item.findtext('Description'))
            CaseQty.append(item.findtext('CaseQty'))
            CaseUOM.append(item.findtext('CaseUOM'))
            PakQty.append(item.findtext('PakQty'))
            PakUOM.append(item.findtext('PakUOM'))
            InventoryGroup.append(item.findtext('InventoryGroup'))

            
# Create a dataframe from the lists created above.
        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    )

Items.drop_duplicates(inplace=True)

Items.reset_index(drop=True, inplace=True)

In [6]:
# creates a new array with unique ItemIds
all_id_list = Items["ItemId"].unique()

In [7]:
# None of the items are egg yolk liq
Items.loc[Items["Description"] == "Egg Yolk Liq"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [8]:
# Sumplemental option for egg, vegan option
Items.loc[Items["ItemId"] == "I-68700"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [9]:
# Gives you the list of breads
breadlist = []


# If the items in the Items list is a "LOAF", "SANDWICH", or "BREAD" then we should add it in the breadlist
# We append it onto the breadlist at position row["ItemId"]
for ind, row in Items.iterrows():
    if ("LOAF" or "SANDWICH" "BREAD") in row["Description"]:
        breadlist.append(row["ItemId"])

breadlist

['I-13004',
 'I-1186',
 'I-1187',
 'I-64492',
 'I-1232',
 'I-14190',
 'I-11664',
 'I-15032',
 'I-12478',
 'I-11842']

In [10]:
Items = Items[~Items["InventoryGroup"].isin(["DISPOSABLES", "SUPPLY"])]

In [11]:
# Based on info below there are 486 rows and 7 columns
Items.shape

(667, 7)

In [12]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [13]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

### Import Ingredients List

In [14]:
# Read ingredients.xml files in the filepath_list and construct a dataframe
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

# Using the Ingredients XML file, we extract attributes containing ingredients, conversion, invFactor, qty, recipe, and uom. 
# Then we append it onto the IngredientId, Coversion, InvFactor, Qty, Recipe, and Uom lists
# Then we create a dataframe using the lists created. 

for filepath in filepath_list:
    path = filepath + '/Ingredients.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Ingredient'):
            IngredientId.append(x.attrib['ingredient'])
            Conversion.append(x.attrib['conversion'])
            InvFactor.append(x.attrib['invFactor'])
            Qty.append(x.attrib['qty'])
            Recipe.append(x.attrib['recipe'])
            Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()
Ingredients.drop_duplicates(subset=["IngredientId", "Recipe"], inplace=True)

Ingredients.reset_index(drop=True, inplace=True)

In [15]:
Ingredients.loc[Ingredients["IngredientId"] == "I-29389"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
1624,I-29389,2.500,Kg,1.00000000,1.0000,P-50739


In [16]:
# This will output the IngredientId on the right side and the number of times the ingredient appears in the Ingredients
# dataframe on the left side. 

# The duplicated() method returns a Series with True and False values that describe which rows in the DataFrame are 
# duplicated and not.

check = Ingredients["IngredientId"].duplicated()

# The line below tells us at which index is the ingredient duplicated. For example I-4598 exists on index 2 of the dataframe and
# then again on index 8. So duplicate is first true on index 8, which is why it is printed below. It also appears in a below 
# index so it is printed again with that index number. 
Ingredients["IngredientId"][check]

15       I-3388
16       I-3642
24       I-6026
27       I-4598
37      I-09837
         ...   
5104    I-17152
5105    I-17675
5106    I-72756
5107     I-7787
5109    P-38661
Name: IngredientId, Length: 3692, dtype: object

In [17]:
# We can see from the printed items above that I-64877 does not appear because it does not get duplicated
Ingredients.loc[Ingredients["IngredientId"] == "I-64877"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [18]:
Ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-48933,1.000,Kg,1.00000000,1.0000,P-10113
1,P-18746,1.000,Kg,1.00000000,1.0000,P-10241
2,I-3388,1.000,L,1.00000000,0.3058,P-10496
3,I-4658,2.270,Kg,2.20462000,0.6942,P-10496
4,I-12176,4.000,g,0.00100000,0.3636,P-12112
...,...,...,...,...,...,...
5107,I-7787,50.000,ml,0.00100000,1.0000,R-73430
5108,I-8281,50.000,g,0.00100000,1.0000,R-73430
5109,P-38661,25.000,mg,0.00000100,1.0000,R-73430
5110,P-73445,150.000,g,0.00100000,1.0000,R-73452


In [19]:
# Below we are checking if we can extract an entire recipe. So we can do .loc with the particular recipe id and then 
# print all the data points (ingredientId, Qty, Uom, Conversion, InvFactor) for that recipe. 
Ingredients.loc[Ingredients["Recipe"] == "R-68698"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [20]:
Items

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-7631,5 SPICE POWDER,1.000,ea,1.000,lb,SPICES
1,I-3617,ALLSPICE GROUND SHAKER,8.000,each,454.000,g,SPICES
2,I-44059,ALMOND BUTTER*,10.000,Kg,1.000,Kg,FOOD - GROCERY
3,I-43652,ANCHO CHILLI,1.200,Kg,1.000,Kg,FOOD - GROCERY
4,I-2853,ANCHOVIES FILLET*,1.700,Kg,1.000,Kg,FOOD - GROCERY
...,...,...,...,...,...,...,...
662,I-72037,YOGURT YOGG*COCONUT VEGAN,3.500,Kg,1.000,Kg,DAIRY
663,I-68111,YOGURT YOGGU*PLAIN COCONUT,3.500,Kg,1.000,Kg,DAIRY
664,I-1490,ZEST FLAVOURS SUGAR ORANG,6.000,lb,1.000,lb,BAKING-RAW INGREDIENTS
665,I-1489,ZEST SUGARED LEMON,5.000,lb,1.000,lb,BAKING-RAW INGREDIENTS


In [21]:
Ingredients.shape

(5112, 6)

In [22]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [23]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

### Import Preps List

In [24]:
# Read preps.xml files in the filepath_list and construct a dataframe
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []


# Here we do the same thing for the Preps XML file where we find the columns using attrib function and then append it onto
# the dataframe called Preps. 

for filepath in filepath_list:
    path = filepath + '/Preps.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prep'):
            PrepId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            PakQty.append(x.findtext('PakQty'))
            PakUOM.append(x.findtext('PakUOM'))
            InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()
preps_columns = Preps.columns
Preps.drop_duplicates(subset=["PrepId"], inplace=True)

Preps.reset_index(drop=True, inplace=True)

In [25]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-61322,AIOLI|Pesto,4.000,L,ZDONT USE OK - PREP
1,P-26184,BAKED|Beans,9.000,Kg,PREP
2,P-54644,BAKED|Moussaka,16.000,PTN,
3,P-54666,BAKED|Pasta|Chicken Alfredo,6.176,Kg,
4,P-54664,BAKED|Pasta|Chorizo Penne,7.360,Kg,
...,...,...,...,...,...
771,P-46079,YIELD|Watermelon*,3.000,Kg,
772,P-41047,YIELD|Yam,1.900,Kg,
773,P-24452,YIELD|Yellow Onion,0.900,lb,
774,P-57145,YIELD|Yellow Pepper,8.300,Kg,


In [26]:
# This tells us that there are not any duplicates in the Preps dataframe. There is a different process|ingredient pair for 
# all the recipes. 
check = Preps["PrepId"].duplicated().any()
print(check)

False


In [27]:
# There are 546 rows and 5 columns
Preps.shape

(776, 5)

In [28]:
# Here we see there is only one place P-50739 is used in the Preps
Preps.loc[Preps["PrepId"] == "P-50739"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup
70,P-50739,COMPOTE|Apple Cinnamon,2.500,L,PREP


In [29]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [30]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

### Import Products List

In [31]:
# Read products.xml files in the filepath_list and construct a dataframe
ProdId = []
Description = []
SalesGroup = []

# From the XML file for products append the id into ProdId, description into Description, and SalesGroup into SalesGroup list.
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe

for filepath in filepath_list:
    path = filepath + '/Products.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prod'):
            ProdId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            SalesGroup.append(x.findtext('SalesGroup'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description, 'SalesGroup': SalesGroup})
Products.drop_duplicates(inplace=True)

Products.reset_index(drop=True, inplace=True)

In [32]:
Products

,ProdId,Description,SalesGroup
0,R-61778,ALF|Flatbread|4 Cheese,OK - AL FORNO
1,R-61780,ALF|Flatbread|Apple & Pancetta,OK - AL FORNO
2,R-61749,ALF|Flatbread|BBQ Chicken,OK - AL FORNO
3,R-61737,ALF|Flatbread|Beef & Onion,OK - AL FORNO
4,R-68122,ALF|Flatbread|Beef Onion|TMRW,OK - AL FORNO
...,...,...,...
337,R-27273,VEG|Toast|OK Avocado,OK - VEGETARIAN KITCHEN
338,R-73241,VEG|Toast|Roasted Tomato,OK- VEGETARIAN KITCHEN FEATURE
339,R-56510,VEG|Toast|Southwest Avocado,OK - VEGETARIAN KITCHEN
340,R-56511,VEG|Toast|Westcoast Avocado,OK - VEGETARIAN KITCHEN


In [33]:
# Here we can see that there is only one R-56966 in the products 
Products.loc[Products["ProdId"] == "R-56966"]

,ProdId,Description,SalesGroup


In [34]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [35]:
# Here we can see that there is only one R-68698 in the products 
Products.loc[Products["ProdId"] == "R-68698"]

,ProdId,Description,SalesGroup


In [36]:
Products.shape

(342, 3)

In [37]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [38]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

### Import Conversions List

In [39]:
# Read conventions.xml files in the filepath_list and construct a dataframe
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

# From the XML file for Conversions append the id into ConversionId, multiplier into Multiplier, ConvertFrom->qty into 
# ConvertFromQty,ConvertFrom->uom into ConvertFromUom, ConvertTo->qty into ConvertToQty and and ConvertTo->uom into the
# CovertToUom list. 
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe


for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Conversion'):
            ConversionId.append(x.attrib['id'])
            Multiplier.append(x.attrib['multiplier'])
            ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
            ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
            ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
            ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [40]:
# Here we can see for example that to convert 1.14 L to 1 L the multiplier is 0.877 since 1/1.14 = 0.877
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L
...,...,...,...,...,...,...
305,I-45114,0.03333333,0.5000,fl oz,15.0000,g
306,I-56382,0.00100000,1.0000,L,1000.0000,g
307,I-65787,0.02000000,1.0000,ea,50.0000,g
308,I-65787,0.03529412,3.0000,oz,85.0000,g


In [41]:
# Here we can check that there are no ingredients listed since 
Conversions.loc[Conversions["ConversionId"] == "I-4582"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [42]:
all_id_list = Items["ItemId"].unique()
all_conv_list = Conversions["ConversionId"].unique()

print("All unique IDs list\n")
print(all_id_list)
print("\n")
print("All unique Conversions list\n")
print(all_conv_list)

All unique IDs list

['I-7631' 'I-3617' 'I-44059' 'I-43652' 'I-2853' 'I-11181' 'I-29389'
 'I-6775' 'I-4461' 'I-4271' 'I-3117' 'I-4971' 'I-4467' 'I-4473' 'I-4472'
 'I-4973' 'I-65741' 'I-27410' 'I-2305' 'I-62928' 'I-19178' 'I-1252'
 'I-1207' 'I-18089' 'I-4495' 'I-4496' 'I-4491' 'I-3118' 'I-3119' 'I-17203'
 'I-6872' 'I-4507' 'I-3130' 'I-3133' 'I-3141' 'I-3619' 'I-72463' 'I-2567'
 'I-3143' 'I-11556' 'I-72464' 'I-57743' 'I-4516' 'I-3144' 'I-7935'
 'I-3145' 'I-3146' 'I-3149' 'I-43544' 'I-3838' 'I-37002' 'I-3846' 'I-3837'
 'I-10869' 'I-4521' 'I-13050' 'I-4288' 'I-4535' 'I-4531' 'I-72753'
 'I-43648' 'I-4538' 'I-41921' 'I-73416' 'I-73415' 'I-31126' 'I-70023'
 'I-32258' 'I-1257' 'I-65951' 'I-72414' 'I-69988' 'I-69728' 'I-8060'
 'I-2074' 'I-60804' 'I-4550' 'I-4555' 'I-4558' 'I-4557' 'I-4556' 'I-4560'
 'I-3159' 'I-3586' 'I-11214' 'I-3620' 'I-4568' 'I-4589' 'I-4572' 'I-11549'
 'I-4574' 'I-4975' 'I-4593' 'I-4594' 'I-56920' 'I-3621' 'I-4598' 'I-4605'
 'I-4599' 'I-39077' 'I-34504' 'I-36383' 'I-7953' '

In [43]:
# Here we have the number of items in the list that are a part of the "all_id_list" but not part of the "all_conv_list"
missing_conv_id = []

for item in all_id_list:
    if item not in all_conv_list:
        missing_conv_id.append(item)
        
missing_conv_id
print(len(missing_conv_id))

509


In [44]:
Conversions.shape

(310, 6)

In [45]:
Conversions.loc[Conversions["ConversionId"] == "I-29389"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [46]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [47]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

***
## Data Summary

In [48]:
# Summary of raw data imported for evaluation
# Here we have a summary of the number of items, preps, ingredients, products, conversions

datasum = pd.DataFrame([Items.shape, Preps.shape, Ingredients.shape, Products.shape, Conversions.shape],
                       columns = ['count', 'columns'], 
                       index = ['Items', 'Preps', 'Ingredients', 'Products', 'Conversions'])
datasum

,count,columns
Items,667,7
Preps,776,5
Ingredients,5112,6
Products,342,3
Conversions,310,6
